In [1]:
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

sys.path.append('../../src')
from paths import FLTRD_DB_DIR, FLLD_DB_DIR
from utils.functions import clean_folder, load_datasets_from_csv
from plot import show_serie, show_gaps

from gaps.plots import show_gaps
from gaps.functions import find_series_gaps, flatten_series_gaps, fill_gaps

In [2]:
clean_folder(FLLD_DB_DIR, "*.csv")

In [3]:
dts = load_datasets_from_csv(FLTRD_DB_DIR).copy()

In [4]:
dts.keys()

dict_keys(['external', 'fan', 'air', 'pump', 'fridge', 'computer', 'house'])

## 1. FILLING LARGE GAPS

In [5]:
gaps_dict = {}
gaps_dict_flat = {}
for key in dts:
    print(key)
    gaps_dict[key] = find_series_gaps(dts[key])
    gaps_dict_flat[key] = flatten_series_gaps(gaps_dict[key])

external
fan
air
pump
fridge
computer
house


In [6]:
for key in dts:
    fill_gaps(dts[key], gaps_dict[key], 30)

In [7]:
for key in dts:
    print(key)
    gaps_dict[key] = find_series_gaps(dts[key])
    gaps_dict_flat[key] = flatten_series_gaps(gaps_dict[key])

external
fan
air
pump
fridge
computer
house


In [8]:
for key in dts:
    dts[key].interpolate(method='from_derivatives', inplace=True)

In [9]:
new_gaps_dict = {}
new_gaps_dict_flat = {}
for key in dts:
    print(key)
    new_gaps_dict[key] = find_series_gaps(dts[key])
    new_gaps_dict_flat[key] = flatten_series_gaps(new_gaps_dict[key])

external
fan
air
pump
fridge
computer
house


In [10]:
for key in dts:
    dts[key].dropna(inplace=True)

In [11]:
(dts['computer'].isna() == False).all()

Pwr    True
H      True
T      True
DI     True
dtype: bool

In [12]:
start_date = '2021-02-17 00:00:00'
stop_date = '2021-03-29 23:59:59'

In [13]:
for key in dts:
    dts[key].name = key
    sel = dts[key].loc[start_date:stop_date]
    sel.to_csv(FLLD_DB_DIR+"/{}.csv".format(key), float_format='%.4f')